# Projeto Final EBAC - Cientista de Dados - Hypothyroid

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from xgboost import XGBClassifier, plot_importance

## Carregar Dados

In [2]:
df = pd.read_csv('Base_M43_Pratique_Hypothyroid.csv')
df.head()

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
0,41,F,f,f,f,f,f,f,f,f,...,t,125,t,1.14,t,109,f,?,SVHC,P
1,23,F,f,f,f,f,f,f,f,f,...,t,102,f,?,f,?,f,?,other,P
2,46,M,f,f,f,f,f,f,f,f,...,t,109,t,0.91,t,120,f,?,other,P
3,70,F,t,f,f,f,f,f,f,f,...,t,175,f,?,f,?,f,?,other,P
4,70,F,f,f,f,f,f,f,f,f,...,t,61,t,0.87,t,70,f,?,SVI,P


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   age                        3772 non-null   object
 1   sex                        3772 non-null   object
 2   on thyroxine               3772 non-null   object
 3   query on thyroxine         3772 non-null   object
 4   on antithyroid medication  3772 non-null   object
 5   sick                       3772 non-null   object
 6   pregnant                   3772 non-null   object
 7   thyroid surgery            3772 non-null   object
 8   I131 treatment             3772 non-null   object
 9   query hypothyroid          3772 non-null   object
 10  query hyperthyroid         3772 non-null   object
 11  lithium                    3772 non-null   object
 12  goitre                     3772 non-null   object
 13  tumor                      3772 non-null   object
 14  hypopitu

In [4]:
df.describe()

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
count,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,...,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772
unique,94,3,2,2,2,2,2,2,2,2,...,2,242,2,147,2,235,1,1,5,2
top,59,F,f,f,f,f,f,f,f,f,...,t,?,t,?,t,?,f,?,other,P
freq,95,2480,3308,3722,3729,3625,3719,3719,3713,3538,...,3541,231,3385,387,3387,385,3772,3772,2201,3481


A análise inicial dos dados revelou algumas questões importantes que precisam ser tratadas. Primeiramente, a maioria das colunas, incluindo variáveis que deveriam ser numéricas como idade e níveis hormonais (TSH, T3, TT4, T4U, FTI, TBG), foram erroneamente classificadas como object. Essa categorização incorreta é um forte indício da presença de valores ausentes ou caracteres não numéricos, como o ponto de interrogação ("?"), que precisam ser devidamente tratados para permitir a conversão para tipos de dados numéricos. Além disso, observou-se que diversas colunas de medição hormonal possuem uma coluna auxiliar (_measured) que indica se o exame foi realizado. Essa dualidade pode ser valiosa para decidir estratégias de imputação de valores ausentes e, potencialmente, servir como uma feature adicional para o treinamento do modelo, indicando se a medição foi ou não efetuada.

Outro ponto crítico identificado é o desequilíbrio na coluna binaryClass, que possui os valores "P" e "N". O valor "P" é predominantemente frequente, representando aproximadamente 92% dos registros (3481 de 3772). Essa disparidade significativa entre as classes pode impactar negativamente o desempenho do modelo preditivo, exigindo a aplicação de técnicas de balanceamento para garantir que o modelo não seja viesado em favor da classe majoritária.